In [1]:
import time
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
resp = Request('https://abbreviations.yourdictionary.com/articles/state-abbrev.html', headers={'User-Agent':'Mozilla'}) #http response
soup1 = BeautifulSoup(urlopen(resp)) #turn respone into readable data

tax_states_dict = {}
for i in soup1.find('ul'):
    if i != ' ':
        temp = str(i).split('>')[1].split('<')[0].split('-')
        tax_states_dict[temp[1].rstrip().lstrip()] = temp[0].rstrip()

tax_states_dict['TX']

'Texas'

In [3]:
def get_net_pay(money, rate, state):
    if rate in ['year', 'month', 'biweek', 'week', 'day', 'hour']:
        url_base = 'https://neuvoo.com/tax-calculator/?iam=&salary=%s&from=%s&region=%s' % (str(money), rate, tax_states_dict[state])
        url_base = '+'.join(url_base.split(' '))
        resp = Request(url_base, headers={'User-Agent':'Mozilla'}) #http response
        soup = BeautifulSoup(urlopen(resp)) #turn respone into readable data
        net_pay = soup.find("div", {"id": "net_pay"}).text
        return net_pay[2:]
    else:
        print('Check your rate')
        return 'Error'

In [4]:
def col_compare(rate, yearly_to, yearly_from, city_to, city_from='Dallas, TX'):
    '''
        rate: string with any value in ['year', 'month', 'biweek', 'week', 'day', 'hour'],
        yearly_to: int,
        yearly_from: int,
        city: string in format "City name, State acronym",
        city_from: string in format "City name, State"
    '''
    #we get the net pay after taxes
    state_to = city_to.split(',')[1].lstrip()
    state_from = city_from.split(',')[1].lstrip()
    net_pay_to = get_net_pay(yearly_to, rate, state_to)
    net_pay_from = get_net_pay(yearly_from, rate, state_from)
    if net_pay_from == 'Error' or net_pay_to == 'Error':
        print('Check your inputs')
    else:
        nw_url = 'https://www.nerdwallet.com/cost-of-living-calculator/compare/%s-vs-%s' % (city_dict[city_from], city_dict[city_to])
        resp = Request(nw_url, headers={'User-Agent':'Mozilla'}) #http response
        key_str = BeautifulSoup(urlopen(resp)).find('div', {'class': 'col-results-header-info'}).text
        
        print('%s after taxes in %s' % (net_pay_from, city_from))
        print('%s after taxes in %s' % (net_pay_to, city_to))
        print('-------------------------------------------------------')
        print(key_str)
        
        key_str = key_str.split(' ')[5:7]
        col_mag = float(key_str[0][:-1])/100
        col_dir = 1+col_mag if key_str[1] == 'higher' else 1-col_mag
        
        col_from = float(''.join(net_pay_from[1:].split(',')))
        col_to = float(''.join(net_pay_to[1:].split(','))) / col_dir
        print('-------------------------------------------------------')
        change_in_col = col_from/col_to
        if change_in_col > 1:
            rud = str(round((change_in_col-1)*100,3)) + '%'
            print('Moving from %s to %s leads to %s reduction in standard of living' % (city_from, city_to, rud))
        else:
            inc = str(round((1-change_in_col)*100, 3)) + '%'
            print('Moving from %s to %s leads to %s increase in standard of living' % (city_from, city_to, inc))
        

In [6]:
f = open('city_urls_nd.txt', 'r')
city_tags = []
for i in f:
    city_tags.append(i[:-1].split('vs')[-1][1:])

city_tags.append('dallas-tx')

def city_formatter(city_tag):
    temp = city_tag.split('-')
    state = temp[-1].upper()
    city = [i.capitalize() for i in temp[:-1]]
    return ' '.join(city) + ', ' + state

city_dict = {city_formatter(i): i for i in city_tags} # Formatted city name so it looks nicer when inputed 

test_city_dict = {i:key for i,key in enumerate(city_dict.keys())} #made so I don't have to type up the city name

In [8]:
test_city_dict

{0: 'Anchorage, AK',
 1: 'Fairbanks, AK',
 2: 'Juneau, AK',
 3: 'Anniston Calhoun County, AL',
 4: 'Auburn Opelika, AL',
 5: 'Birmingham, AL',
 6: 'Decatur Hartselle, AL',
 7: 'Dothan, AL',
 8: 'Florence, AL',
 9: 'Huntsville, AL',
 10: 'Mobile, AL',
 11: 'Montgomery, AL',
 12: 'Conway, AR',
 13: 'Fayetteville, AR',
 14: 'Hot Springs, AR',
 15: 'Jonesboro, AR',
 16: 'Little Rock North Little Rock, AR',
 17: 'Bullhead City, AZ',
 18: 'Flagstaff, AZ',
 19: 'Lake Havasu City, AZ',
 20: 'Phoenix, AZ',
 21: 'Surprise, AZ',
 22: 'Tucson, AZ',
 23: 'Yuma, AZ',
 24: 'Bakersfield, CA',
 25: 'Los Angeles Long Beach, CA',
 26: 'Modesto, CA',
 27: 'Oakland, CA',
 28: 'Orange County, CA',
 29: 'Sacramento, CA',
 30: 'San Diego, CA',
 31: 'San Francisco, CA',
 32: 'Stockton, CA',
 33: 'Colorado Springs, CO',
 34: 'Denver, CO',
 35: 'Glenwood Springs, CO',
 36: 'Pueblo, CO',
 37: 'Westminster, CO',
 38: 'New Haven, CT',
 39: 'Stamford, CT',
 40: 'Washington, DC',
 41: 'Dover, DE',
 42: 'Sussex County

In [13]:
col_compare('year', 70000, 70000, test_city_dict[244])

$55,946 after taxes in Dallas, TX
$52,524 after taxes in Richmond, VA
-------------------------------------------------------
The cost of living is 13% lower in Richmond, VA
-------------------------------------------------------
Moving from Dallas, TX to Richmond, VA leads to 7.332% increase in standard of living


In [109]:
col_compare('hour', 23, 45, test_city_dict[6], test_city_dict[8])

$33.29 after taxes in Florence, AL
$18.39 after taxes in Decatur Hartselle, AL
-------------------------------------------------------
The cost of living is 2% lower in Decatur-Hartselle, AL
-------------------------------------------------------
Moving from Florence, AL to Decatur Hartselle, AL leads to 77.402% reduction in standard of living


In [14]:
col_compare('year', 90000, 90000, test_city_dict[159])

$70,016 after taxes in Dallas, TX
$65,022 after taxes in New York Manhattan, NY
-------------------------------------------------------
The cost of living is 128% higher in New York (Manhattan), NY
-------------------------------------------------------
Moving from Dallas, TX to New York Manhattan, NY leads to 145.511% reduction in standard of living


### Steps for extracting city values
1. Change tabs to nerd wallet
2. Click on bar with "I want to live in:"
3. Press 'Enter' the first time then everytime after press 'Down' + 'Enter'
4. Go to URL address bar and copy it
5. Go to excel and paste it and press 'Enter'
6. Then go back to nerd wallet page and repeat at step 2


In [132]:
import pyautogui as pg

In [169]:
tab = pg.position() #set up the postion for the tab

In [171]:
bar = pg.position() #set up the search bar position

In [172]:
url = pg.position() #set up the url address position

In [173]:
excel = pg.position() #set up the excel position

In [185]:
# First pass b/c we can't press down the first time
pg.moveTo(tab)
pg.click()
pg.moveTo(bar)
pg.click()
pg.press('enter')
pg.moveTo(url)
pg.click()
pg.hotkey('ctrl', 'a')
pg.hotkey('ctrl', 'c')
pg.moveTo(excel)
pg.click()
pg.hotkey('ctrl', 'v')
pg.press('enter')

In [186]:
def add_url_to_excel():
    pg.moveTo(tab)
    pg.click()
    pg.moveTo(bar)
    pg.click()
    pg.press('down')
    pg.press('enter')
    pg.moveTo(url)
    pg.click()
    pg.hotkey('ctrl', 'a')
    pg.hotkey('ctrl', 'c')
    pg.moveTo(excel)
    pg.click()
    pg.hotkey('ctrl', 'v')
    pg.press('enter')

In [187]:
for i in range(266): #316 total minus 50 states
    add_url_to_excel()